In [1]:
import pandas as pd
import numpy as np

In [22]:
Rainfall_data=pd.read_excel(r"E:\Agriculture project\Data Processing\filtered_rainfall_analysis.xlsx")
Soil_moisture_data=pd.read_excel(r"E:\Agriculture project\Data\Semi-prepared data\Soilmoisture.xlsx")
Temperature_data=pd.read_excel(r"E:\Agriculture project\Data\Semi-prepared data\Temperature_wide.xlsx")

In [23]:
Rainfall_data.columns


Index(['Date', 'Nyeri', 'Meru', 'Kirinyaga', 'Embu', 'Murang'a',
       'Elgeyo-Marakwet', 'Kericho', 'Bomet', 'Kiambu', 'Nandi', 'Nyandarua',
       'Kisii', 'Trans Nzoia', 'Uasin Gishu', 'Bungoma', 'Kakamega', 'Vihiga',
       'Nakuru', 'Laikipia', 'Kitui', 'Machakos', 'Makueni', 'Tharaka-Nithi',
       'West Pokot', 'Narok', 'Baringo'],
      dtype='object')

In [24]:
Soil_moisture_data.columns


Index(['Date', 'Baringo', 'Bomet', 'Bungoma', 'Busia', 'Elgeyo-Marakwet',
       'Embu', 'Garissa', 'Homa Bay', 'Isiolo', 'Kajiado', 'Kakamega',
       'Kericho', 'Kiambu', 'Kilifi', 'Kirinyaga', 'Kisii', 'Kisumu', 'Kitui',
       'Kwale', 'Laikipia', 'Lamu', 'Machakos', 'Makueni', 'Mandera',
       'Marsabit', 'Meru', 'Migori', 'Mombasa', 'Murang'a', 'Nairobi',
       'Nakuru', 'Nandi', 'Narok', 'Nyamira', 'Nyandarua', 'Nyeri', 'Samburu',
       'Siaya', 'Taita Taveta', 'Tana River', 'Tharaka-Nithi', 'Trans Nzoia',
       'Turkana', 'Uasin Gishu', 'Vihiga', 'Wajir', 'West Pokot'],
      dtype='object')

<h2> Merge All the dataset into one Excel file </h2>

In [ ]:
# List of selected counties in Agro-Ecological Zones 1-4
selected_counties = [
    "Nyeri", "Meru", "Embu", "Murang'a",
    "Kericho", "Bomet", "Nandi", "Nyandarua", "Kisii",
    "Uasin Gishu", "Bungoma", "Kakamega",
    "Nakuru", "Laikipia", "Kitui", "Machakos", "Makueni", "Tharaka-Nithi",
    "West Pokot", "Narok", "Baringo"
]

# Function to preprocess data: rename Date column, convert to datetime, and filter counties
def preprocess_data(df, name):
    df.rename(columns={df.columns[0]: "Date"}, inplace=True)
    df["Date"] = pd.to_datetime(df["Date"])
    df = df[df["Date"].dt.year >= 1984]  # Remove data before 1984
    df = df[["Date"] + selected_counties]  # Select only relevant counties
    print(f"{name} Data Loaded | Shape: {df.shape} | Date Range: {df['Date'].min()} to {df['Date'].max()}")
    return df

# Preprocess all datasets
Rainfall_data = preprocess_data(Rainfall_data, "Rainfall")
Soil_moisture_data = preprocess_data(Soil_moisture_data, "Soil Moisture")
Temperature_data = preprocess_data(Temperature_data, "Temperature")

# Check common dates before merging
common_dates = (
    set(Rainfall_data["Date"]) & 
    set(Soil_moisture_data["Date"]) & 
    set(Temperature_data["Date"])
)
print(f"Common Dates Count: {len(common_dates)}")

# Merge datasets and ensure Temperature columns get "_temp" suffix
merged_data = Rainfall_data.merge(Soil_moisture_data, on="Date", suffixes=("_rain", "_soil"))
merged_data = merged_data.merge(Temperature_data, on="Date", how="inner", suffixes=("", "_temp"))

# Manually add "_temp" suffix to Temperature columns
for county in selected_counties:
    if county in merged_data.columns:
        merged_data.rename(columns={county: f"{county}_temp"}, inplace=True)

# Save to Excel
output_path = r"E:\Agriculture project\Analysis\Merged_data.xlsx"
merged_data.to_excel(output_path, index=False)
print(f"Data Merged Successfully and saved to {output_path}")


Rainfall Data Loaded | Shape: (489, 22) | Date Range: 1984-01-01 00:00:00 to 2024-09-01 00:00:00
Soil Moisture Data Loaded | Shape: (489, 22) | Date Range: 1984-01-01 00:00:00 to 2024-09-01 00:00:00
Temperature Data Loaded | Shape: (468, 22) | Date Range: 1984-01-01 00:00:00 to 2022-12-01 00:00:00
Common Dates Count: 468


C:\Users\Rono\AppData\Local\Temp\ipykernel_3960\2493766760.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={df.columns[0]: "Date"}, inplace=True)
C:\Users\Rono\AppData\Local\Temp\ipykernel_3960\2493766760.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Date"] = pd.to_datetime(df["Date"])


Data Merged Successfully and saved to E:\Agriculture project\Data Processing\Merged_data.xlsx
